In [10]:
import pandas as pd
import numpy as np
from pmdarima import auto_arima
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error
import os


# Using energy Source Datasets

# Source of Energy Coal

In [11]:
# Read the Excel file
# Data_Status: Indicates the status of the data. The value "2020F" suggests that it is a forecast for the year 2020.
# State: Represents the state for which the data is recorded (in this case, "CA" for California).
# MSN: Stands for "Monthly State Names" and refers to the specific energy metric or variable being measured. Examples include ARICD, ARICV, ARTCD, ARTCV, ARTXD, WWTXV, WXICD, WXICV, ZWCDP, ZWHDP.
df = pd.read_excel('Datasets/use_tot_sector.xlsx' , sheet_name='Transportation Sector')


In [12]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,27139,31895,34182,32387,32246,34379,36233,43862,47177,...,184846,167929,151923,155043,162888,153572,155868,163804,164258,164595
1,AL,176015,172023,179802,189395,190545,200727,206854,216031,242554,...,478191,472022,466359,465284,480784,504527,498739,488594,500768,497755
2,AR,104652,114214,116715,123586,130214,129593,136348,139183,153822,...,287138,277918,276347,280150,276984,283503,285148,286192,288865,271075
3,AZ,116689,121239,129736,136740,143720,139441,148689,159735,183623,...,513043,502603,510887,516809,526892,539916,543230,554065,569796,507885
4,CA,1224448,1292718,1316805,1354256,1421236,1491493,1592738,1654402,1779333,...,2828580,2773267,2777173,2798542,2865537,2970922,3049442,3062408,3059749,2355476


# Data statistics

In [13]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,27139,31895,34182,32387,32246,34379,36233,43862,47177,...,184846,167929,151923,155043,162888,153572,155868,163804,164258,164595
1,AL,176015,172023,179802,189395,190545,200727,206854,216031,242554,...,478191,472022,466359,465284,480784,504527,498739,488594,500768,497755
2,AR,104652,114214,116715,123586,130214,129593,136348,139183,153822,...,287138,277918,276347,280150,276984,283503,285148,286192,288865,271075
3,AZ,116689,121239,129736,136740,143720,139441,148689,159735,183623,...,513043,502603,510887,516809,526892,539916,543230,554065,569796,507885
4,CA,1224448,1292718,1316805,1354256,1421236,1491493,1592738,1654402,1779333,...,2828580,2773267,2777173,2798542,2865537,2970922,3049442,3062408,3059749,2355476


In [14]:
df.describe()

,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
count,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,...,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01,5.200000e+01
mean,4.075621e+05,4.142041e+05,4.315256e+05,4.482716e+05,4.614980e+05,4.782312e+05,5.038746e+05,5.289042e+05,5.717197e+05,5.963359e+05,...,1.022973e+06,1.004818e+06,1.023597e+06,1.033588e+06,1.047883e+06,1.068934e+06,1.077777e+06,1.095029e+06,1.099603e+06,9.365031e+05
std,1.458330e+06,1.482547e+06,1.543791e+06,1.603427e+06,1.651209e+06,1.711306e+06,1.803153e+06,1.892624e+06,2.045393e+06,2.133330e+06,...,3.657982e+06,3.593404e+06,3.662617e+06,3.697785e+06,3.749614e+06,3.825952e+06,3.858503e+06,3.920710e+06,3.937812e+06,3.349842e+06
min,1.928500e+04,1.890300e+04,1.945100e+04,1.972000e+04,2.018600e+04,2.115600e+04,2.264800e+04,2.425200e+04,2.560200e+04,2.660400e+04,...,2.121100e+04,1.911200e+04,1.942000e+04,2.090200e+04,2.027600e+04,2.125000e+04,1.844100e+04,2.030000e+04,2.180200e+04,1.680600e+04
25%,6.382775e+04,6.802425e+04,6.605675e+04,6.872825e+04,7.004450e+04,7.018450e+04,7.797575e+04,7.671525e+04,8.262475e+04,8.627200e+04,...,1.800680e+05,1.729518e+05,1.682105e+05,1.669775e+05,1.709782e+05,1.777265e+05,1.717152e+05,1.877818e+05,1.825965e+05,1.686442e+05
50%,1.508525e+05,1.504585e+05,1.593585e+05,1.665300e+05,1.694355e+05,1.716140e+05,1.817260e+05,1.912320e+05,2.072475e+05,2.158670e+05,...,4.318325e+05,4.278915e+05,4.296560e+05,4.455535e+05,4.491110e+05,4.416985e+05,4.396955e+05,4.595395e+05,4.630705e+05,3.824985e+05
75%,2.545665e+05,2.542460e+05,2.608520e+05,2.708550e+05,2.851062e+05,3.011890e+05,3.079172e+05,3.234212e+05,3.542462e+05,3.717890e+05,...,6.543645e+05,6.258250e+05,6.346335e+05,6.456502e+05,6.512528e+05,7.042748e+05,6.770618e+05,6.832680e+05,6.870220e+05,6.085405e+05
max,1.059661e+07,1.076930e+07,1.121966e+07,1.165506e+07,1.199895e+07,1.243401e+07,1.310074e+07,1.375151e+07,1.486471e+07,1.550474e+07,...,2.659729e+07,2.612528e+07,2.661351e+07,2.687330e+07,2.724496e+07,2.779228e+07,2.802220e+07,2.847075e+07,2.858969e+07,2.434908e+07


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 62 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   State   52 non-null     object
 1   1960    52 non-null     int64 
 2   1961    52 non-null     int64 
 3   1962    52 non-null     int64 
 4   1963    52 non-null     int64 
 5   1964    52 non-null     int64 
 6   1965    52 non-null     int64 
 7   1966    52 non-null     int64 
 8   1967    52 non-null     int64 
 9   1968    52 non-null     int64 
 10  1969    52 non-null     int64 
 11  1970    52 non-null     int64 
 12  1971    52 non-null     int64 
 13  1972    52 non-null     int64 
 14  1973    52 non-null     int64 
 15  1974    52 non-null     int64 
 16  1975    52 non-null     int64 
 17  1976    52 non-null     int64 
 18  1977    52 non-null     int64 
 19  1978    52 non-null     int64 
 20  1979    52 non-null     int64 
 21  1980    52 non-null     int64 
 22  1981    52 non-null     int6

In [16]:
df.head()

,State,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,AK,27139,31895,34182,32387,32246,34379,36233,43862,47177,...,184846,167929,151923,155043,162888,153572,155868,163804,164258,164595
1,AL,176015,172023,179802,189395,190545,200727,206854,216031,242554,...,478191,472022,466359,465284,480784,504527,498739,488594,500768,497755
2,AR,104652,114214,116715,123586,130214,129593,136348,139183,153822,...,287138,277918,276347,280150,276984,283503,285148,286192,288865,271075
3,AZ,116689,121239,129736,136740,143720,139441,148689,159735,183623,...,513043,502603,510887,516809,526892,539916,543230,554065,569796,507885
4,CA,1224448,1292718,1316805,1354256,1421236,1491493,1592738,1654402,1779333,...,2828580,2773267,2777173,2798542,2865537,2970922,3049442,3062408,3059749,2355476


# Transformation of Data

In [17]:
df_trans = df.melt(id_vars=['State'] , var_name='Year', value_name='Yearly Data')
df_trans['Year'] = pd.to_datetime(df_trans['Year'], format='%Y')

df_trans


,State,Year,Yearly Data
0,AK,1960-01-01,27139
1,AL,1960-01-01,176015
2,AR,1960-01-01,104652
3,AZ,1960-01-01,116689
4,CA,1960-01-01,1224448
...,...,...,...
3167,WA,2020-01-01,504756
3168,WI,2020-01-01,404102
3169,WV,2020-01-01,169994
3170,WY,2020-01-01,107045


# Modeling the Data
## Using ARIMA Model

In [18]:
os.makedirs('Plots/use_tot_Sector_data_Transportation_Sector/Arima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Transportation Sector : Energy Consumption Forecast State : {State} using ARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_tot_Sector_data_Transportation_Sector/Arima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        

Mean Absolute Error (MAE): 5719.51
Mean Squared Error (MSE): 51645250.46
Mean Absolute Percentage Error (MAPE): 3.65


Mean Absolute Error (MAE): 9360.86
Mean Squared Error (MSE): 115244003.83
Mean Absolute Percentage Error (MAPE): 1.88


Mean Absolute Error (MAE): 5540.56
Mean Squared Error (MSE): 96205874.14
Mean Absolute Percentage Error (MAPE): 2.02


Mean Absolute Error (MAE): 16230.96
Mean Squared Error (MSE): 679271960.65
Mean Absolute Percentage Error (MAPE): 3.11


Mean Absolute Error (MAE): 208248.82
Mean Squared Error (MSE): 94576306843.72
Mean Absolute Percentage Error (MAPE): 8.11


...

Mean Absolute Error (MAE): 58130.05
Mean Squared Error (MSE): 5507167436.82
Mean Absolute Percentage Error (MAPE): 10.1


Mean Absolute Error (MAE): 19224.14
Mean Squared Error (MSE): 816404926.74
Mean Absolute Percentage Error (MAPE): 4.62


Mean Absolute Error (MAE): 8663.58
Mean Squared Error (MSE): 88521843.81
Mean Absolute Percentage Error (MAPE): 4.78


Mean Absolute Error (MAE): 2690.17
Mean Squared Error (MSE): 23772776.61
Mean Absolute Percentage Error (MAPE): 2.48


Mean Absolute Error (MAE): 1055496.02
Mean Squared Error (MSE): 3930147590459.01
Mean Absolute Percentage Error (MAPE): 4.24


## Using SARIMA Model

In [19]:
os.makedirs('Plots/use_tot_Sector_data_Transportation_Sector/Sarima_results_plots',exist_ok=True)

for State in df_trans['State'].unique():
        try:
            
            fig = go.Figure()

            # Get the energy consumption data for the current country and sector
            df_filter = df_trans[df_trans['State'] == State ][['Year', 'Yearly Data']]
            df_filter_index = df_filter.set_index('Year')

            train_data = df_filter[:-5]
            test_data = df_filter[-5:]
            
            # Prepare the data for modeling
            years = df_filter_index.index
            energy_consumption = df_filter_index.values.flatten()

                    # Split the data into training and testing
            # Use all data except the last 5 years for training
            Horizan = -5
            train_data = energy_consumption[:Horizan]
            test_data = energy_consumption[Horizan:]  # Use the last 5 years for testing

            # Fit the auto ARIMA model
            model = auto_arima(train_data, seasonal=False)
            model.fit(train_data)

            # Generate predictions
            predictions = model.predict(n_periods=len(test_data))
            predictions_ahead_in_future = model.predict(n_periods=len(test_data)+15)

            # Calculate evaluation metrics
            mae = mean_absolute_error(test_data, predictions)
            mse = mean_squared_error(test_data, predictions)
            mape = np.mean(np.abs((test_data - predictions) / test_data)) * 100

            print('Mean Absolute Error (MAE):', np.round(mae,2))
            print('Mean Squared Error (MSE):', np.round(mse,2))
            print('Mean Absolute Percentage Error (MAPE):', np.round(mape,2))
            
            # Plot the training data
            fig.add_trace(go.Scatter(
                x=years[:Horizan], y=train_data, mode='lines+markers', name='Training Data'))

            # Plot the predictions
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=test_data, mode='lines+markers', name='Actual'))
            fig.add_trace(go.Scatter(
                x=years[Horizan:], y=predictions, mode='lines+markers', name='Predicted'))

            fig.add_trace(go.Scatter(
                x=pd.date_range(start = years[Horizan],periods=15,freq='Y'), y=predictions_ahead_in_future, mode='lines+markers', name='Prediction till 2030'))

            # Update the layout
            fig.update_layout(title=f'Transportation Sector : Energy Consumption Forecast State : {State} using SARIMA Model',
                            xaxis_title='Year', yaxis_title='Energy Consumption')

            # Show the plot
            fig.show()
            # print(State,msn)
            fig.write_image(f'Plots/use_tot_Sector_data_Transportation_Sector/Sarima_results_plots/{State}.png')
            # break
        except:
            print('Error occoured in Combination State : {} and MSN : {} Due NaN Value'.format(State,mse))
        # break

Mean Absolute Error (MAE): 5719.51
Mean Squared Error (MSE): 51645250.46
Mean Absolute Percentage Error (MAPE): 3.65


Mean Absolute Error (MAE): 9360.86
Mean Squared Error (MSE): 115244003.83
Mean Absolute Percentage Error (MAPE): 1.88


Mean Absolute Error (MAE): 5540.56
Mean Squared Error (MSE): 96205874.14
Mean Absolute Percentage Error (MAPE): 2.02


Mean Absolute Error (MAE): 16230.96
Mean Squared Error (MSE): 679271960.65
Mean Absolute Percentage Error (MAPE): 3.11


Mean Absolute Error (MAE): 208248.82
Mean Squared Error (MSE): 94576306843.72
Mean Absolute Percentage Error (MAPE): 8.11


...

Mean Absolute Error (MAE): 58130.05
Mean Squared Error (MSE): 5507167436.82
Mean Absolute Percentage Error (MAPE): 10.1


Mean Absolute Error (MAE): 19224.14
Mean Squared Error (MSE): 816404926.74
Mean Absolute Percentage Error (MAPE): 4.62


Mean Absolute Error (MAE): 8663.58
Mean Squared Error (MSE): 88521843.81
Mean Absolute Percentage Error (MAPE): 4.78


Mean Absolute Error (MAE): 2690.17
Mean Squared Error (MSE): 23772776.61
Mean Absolute Percentage Error (MAPE): 2.48


Mean Absolute Error (MAE): 1055496.02
Mean Squared Error (MSE): 3930147590459.01
Mean Absolute Percentage Error (MAPE): 4.24
